In [86]:
# Importing torch and setting the device
import torch

# Setting GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Properties
print(torch.cuda.get_device_name())
print(torch.cuda.get_device_capability()) # torch.cuda.get_device_properties()

NVIDIA GeForce MX350
(6, 1)


*A PyTorch computational graph is a Directed Acyclic Graph (DAG) that represents the sequence of operations performed on tensors during the forward pass of a model. This graph is dynamically built during runtime, meaning it is constructed on the fly as operations are executed.*

#### AutoGrad for Scalers

In [87]:
x = torch.tensor(3.0, requires_grad=True, device=device) # Enables the differentation wrt this variable (x)
x

tensor(3., device='cuda:0', requires_grad=True)

In [88]:
y = x**2
y

tensor(9., device='cuda:0', grad_fn=<PowBackward0>)

*`backward` - Computes the gradient of current tensor wrt graph leaves. The graph is differentiated using the chain rule.*

In [89]:
y.backward(retain_graph = True)
# Computational Graph: X -> square function -> y(leaf node)

In [90]:
x.grad
# Differentation of leaf node wrt x

tensor(6., device='cuda:0')

*Note: When you run `y.backward()` again (only if `retain_graph = True`) then PyTorch saves the gradient in its memory and add it when you run it again.*

In [91]:
x = torch.tensor(4.0, requires_grad=True)
y = 2 * x * (torch.cos(x) ** 2) # Multiplication Rule of Derivative
z = torch.sin(y)

In [92]:
x

tensor(4., requires_grad=True)

In [93]:
y

tensor(3.4180, grad_fn=<MulBackward0>)

In [94]:
z

tensor(-0.2729, grad_fn=<SinBackward0>)

`Computational Graph`: X    —→    func(2 * x * cos(x)) = y    —→    func(sin(y)) = z

In [95]:
z.backward()

In [96]:
x.grad
# y.grad - You only able to calculate the gradient wrt leaf node (the last variable) only

tensor(6.7924)

*Gradients of non leaf node(z) will not be calculated*

---

#### Basic AutoGrad from scratch

In [97]:
# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [98]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [99]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012)

In [100]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

# Chain Rule
dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [101]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [102]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [103]:
# Forward Pass
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [104]:
# Predict Probabilities
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [105]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

`Computational Graph`: w, b    —→    foreard pass(x, w, b) = z    —→    probabilities(z) = y_pred    —→    loss(y_pred, y_actual)

In [106]:
loss.backward()

In [107]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


---

#### AutoGrad for Tensors

In [108]:
# Gradients with multiple X values
x = torch.tensor([[1.0, 2.0, 3.0], [2.5, 9.8, 5.7]], requires_grad = True)
x

tensor([[1.0000, 2.0000, 3.0000],
        [2.5000, 9.8000, 5.7000]], requires_grad=True)

In [109]:
y = (x**2).mean()
y

tensor(24.7967, grad_fn=<MeanBackward0>)

In [110]:
y.backward()

In [111]:
x.grad

tensor([[0.3333, 0.6667, 1.0000],
        [0.8333, 3.2667, 1.9000]])

---

#### Gradient Accumulation

In [112]:
# clearing grad
x = torch.tensor(2.0, requires_grad = True)
x

tensor(2., requires_grad=True)

In [113]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [114]:
y.backward(retain_graph=True)

In [115]:
x.grad

tensor(4.)

*The problem with executing `y.backward()` and `x.grad` muptiple times will store the gradient values in the memory → If you run this more than one time then ouptut will be `d1 + d2 + ...` - The values of gradients will get added*

In [116]:
# Inplace operation
x.grad.zero_()

tensor(0.)

---

#### Disable gradient tracking

In [117]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2

y.backward(retain_graph=True)
x.grad

tensor(4.)

In [118]:
# Way 1: requires_grad_ - perminantly disable the gradients
x.requires_grad_(False) # requires_grad = False (_ inplace)

tensor(2.)

In [119]:
y = x ** 2 # Calculating y again after updating the state of x

try:
    y.backward()
except Exception as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [120]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2

y.backward()
x.grad

tensor(4.)

In [121]:
# Way 2: detach() - The variable gets detached from the DAG
z = x.detach() # Copy the value of x but its requires_grad becomes false
z

tensor(2.)

In [122]:
y = z ** 2
y

tensor(4.)

In [123]:
try:
    y.backward()
except Exception as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [124]:
# Consider you are training your neural network
x = torch.tensor([1.0, 2.0], requires_grad=True) # Only Tensors of floating point and complex dtype can require gradients
x = torch.tensor(2.4, requires_grad=True)
y = x ** 2
y.backward()
print(x.grad)
# The neural network is trained

# Its prediction time
with torch.no_grad():
    # In this scope only, all the parameter's requires_grad parameter becomes false
    y = x + 10

    try:
        y.backward()
        print(x.grad())
    except Exception as e:
        print(e)

# For an instance, need to run a block of code without building a DAG, then you can stop the gradients by using context manager

tensor(4.8000)
element 0 of tensors does not require grad and does not have a grad_fn


*`torch.no_grad()` is the most recommended way to stop calculating or tracking grad*